In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
############################# 1.Importation des bibliothèques #############################

#Pour l'interaction avec le système d'exploitation
import os
import cv2
#Pour manipuler les tableaux
import numpy as np

#Encoding and Split data into Train/Test Sets
#Pour encoder les étiquettes de classe
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Tensorflow Keras CNN Model
#pour la création et l'entraînement du modèle CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

#Plot Images
#Pour afficher des images
import matplotlib.pyplot as plt


############################# 2.Chargement et prétraitement des images #############################

#Définition du répertoire contenant les images de fleurs
folder_dir = '/content/drive/MyDrive/flowers'

#Initialisation de listes pour stocker les données (images) et les étiquettes (labels)
data = []
label = []

SIZE = 128 #Crop the image to 128x128

# Cette partie lit les images d'un répertoire spécifié, les convertit en format RGB,
# les redimensionne à une taille définie (128x128 pixels), et les stocke dans data.
# Les étiquettes correspondantes sont stockées dans la liste label.
for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE,SIZE))
            data.append(im)
        else:
            continue

# Conversion des listes en tableaux NumPy
data_arr = np.array(data)
label_arr = np.array(label)

############################# 3.Encodage des étiquettes et division des données #############################

# Les étiquettes sont encodées en nombres entiers, puis en vecteurs binaires
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
y = to_categorical(y,5)
# Les valeurs des pixels des images sont normalisées en les divisant par 255
X = data_arr/255
# Séparation des données en ensembles d'entraînement (X_train, y_train) et de test (X_test, y_test) à l'aide de train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

############################# 4.Création du modèle CNN #############################

#Ce code définit la structure d'un modèle de réseau de neurones convolutionnel (CNN) à l'aide de l'interface séquentielle de Keras,
#qui est un composant de TensorFlow.

#Ces lignes de code définissent l’architecture du modèle CNN.

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (SIZE,SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

#Le modèle est composé de plusieurs couches de convolution et de pooling, suivies d’une couche de flattening et de couches fully connected
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
#La dernière couche utilise une activation softmax pour produire une distribution de probabilités sur les 5 catégories
model.add(Dense(5, activation = "softmax"))

############################# 5.Augmentation des données #############################
# Ces lignes de code créent un générateur de données qui applique des transformations aléatoires aux images
# d’entraînement pour augmenter la diversité des données
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

############################# 6.Compilation et entraînement du modèle #############################

#le modèle est compilé avec l’optimiseur Adam, la perte de catégorie croisée catégorique, et la métrique de précision.
#Le modèle est ensuite entraîné en utilisant le générateur de données.

model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=32
epochs=64

history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),
                              verbose = 1)

############################# 7.Visualisation des résultats #############################

#Ces lignes de code affichent un échantillon d’images de test avec les étiquettes réelles et prédites.
#Si la prédiction est correcte, le titre et le sous-titre sont affichés en vert, sinon en rouge.
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))
for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')

Output hidden; open in https://colab.research.google.com to view.